<a href="https://colab.research.google.com/github/Turtle-Grace/gracehuangtw/blob/main/0325_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Predicting Customer Churn with a Neural Network

You are provided with a real-world dataset from a telecom company that includes information about their customers, such as services signed up for, contract type, and payment behavior. Your task is to build a simple neural network to predict whether a customer will churn (i.e., leave the company) or not.

## Task
Build, train, and evaluate a neural network that predicts whether a customer has churned (`Churn` = Yes) or not (`Churn` = No), based on the other available features in the dataset.

1. **Drop irrelevant columns**, which have no numerical meaning.
2. **Convert columns** to numeric. There may be non-numeric entries; handle them appropriately (e.g. by coercing errors and dropping missing values).
3. **Convert the target column** from categorical ("Yes"/"No") to binary (1/0).
4. **Encode categorical features** using the `get_dummies`-function.
5. **Prepare your data** before you feed it into your model.
6. **Build a neural network** using Keras (e.g., with 1–3 layers) to predict churn.
7. **Compile your model** with an appropriate loss function and optimizer.
8. **Train your model** on the training data.
9. **Evaluate the performance** of your model on the test set and report the test accuracy.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

df = pd.read_csv("Telco-Customer-Churn.csv")
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


(None,
    customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
 0  7590-VHVEG  Female              0     Yes         No       1           No   
 1  5575-GNVDE    Male              0      No         No      34          Yes   
 2  3668-QPYBK    Male              0      No         No       2          Yes   
 3  7795-CFOCW    Male              0      No         No      45           No   
 4  9237-HQITU  Female              0      No         No       2          Yes   
 
       MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
 0  No phone service             DSL             No  ...               No   
 1                No             DSL            Yes  ...              Yes   
 2                No             DSL            Yes  ...               No   
 3  No phone service             DSL            Yes  ...              Yes   
 4                No     Fiber optic             No  ...               No   
 
   TechSupport StreamingTV StreamingMovie

In [3]:
#1. Drop irrelevant columns, which have no numerical meaning.
df = df.drop(columns=['customerID'])

#2. Convert 'TotalCharges' to numeric, coerce errors
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

#Drop rows with any missing values (after coercion)
df = df.dropna()

#3. Convert target column 'Churn' to binary
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

#4. One-hot encode categorical variables
df_encoded = pd.get_dummies(df, drop_first=True)

#5. Prepare features and target
X = df_encoded.drop(columns=['Churn'])
y = df_encoded['Churn']
#Standardise features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 8: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5625, 30), (1407, 30), (5625,), (1407,))

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
#6. Build the neural network model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

#7. Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])

#8. Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

#9. Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
test_accuracy

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0.7846481800079346

In [5]:
#A model that doesn't require TensorFlow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#Train a logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

#Predict on the test set
y_pred = log_reg.predict(X_test)

#Evaluate model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

accuracy, report, conf_matrix

(0.7874911158493249,
 '              precision    recall  f1-score   support\n\n           0       0.83      0.89      0.86      1033\n           1       0.62      0.52      0.56       374\n\n    accuracy                           0.79      1407\n   macro avg       0.73      0.70      0.71      1407\nweighted avg       0.78      0.79      0.78      1407\n',
 array([[915, 118],
        [181, 193]]))

The model is good at predicting customers who do not churn.
But it struggles with detecting customers who do churn, which is common with imbalanced classes.
[[914, 119],   ← 914 true negatives, 119 false positives
 [181, 193]]   ← 181 false negatives, 193 true positives